In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ks_2samp
import pandas as pd
import pickle

real_data = pd.read_csv('../../data/Cancer_Data_new_cols.csv')
real_data.drop(columns=['id'], inplace=True)

diagnosis_le = LabelEncoder()
ethnicity_le = LabelEncoder()
real_data['diagnosis'] = diagnosis_le.fit_transform(real_data['diagnosis'].astype(str))
real_data['ethnicity'] = ethnicity_le.fit_transform(real_data['ethnicity'].astype(str))

ss = StandardScaler()
real_data = pd.DataFrame(ss.fit_transform(real_data), columns=real_data.columns, index=real_data.index)

In [3]:
# Define parameters
n_features = len(real_data.columns)
latent_dim = 7

# Encoder
input_data = Input(shape=(n_features,), name="Input_Layer")
encoder = Dense(64, activation="relu", name="Encoder_Hidden_1")(input_data)
encoder = Dense(32, activation="relu", name="Encoder_Hidden_2")(encoder)
latent_space = Dense(latent_dim, activation="linear", name="Latent_Space")(encoder)

# Decoder
decoder = Dense(32, activation="relu", name="Decoder_Hidden_1")(latent_space)
decoder = Dense(64, activation="relu", name="Decoder_Hidden_2")(decoder)
output_data = Dense(n_features, activation="linear", name="Output_Layer")(decoder)

# Autoencoder Model
autoencoder = Model(inputs=input_data, outputs=output_data, name="Autoencoder")
autoencoder.compile(optimizer="adam", loss="mse")

# Encoder Model
encoder_model = Model(inputs=input_data, outputs=latent_space, name="Encoder")

# Decoder Model
decoder_input = Input(shape=(latent_dim,), name="Decoder_Input")
decoder_layer1 = autoencoder.get_layer("Decoder_Hidden_1")(decoder_input)
decoder_layer2 = autoencoder.get_layer("Decoder_Hidden_2")(decoder_layer1)
decoder_output = autoencoder.get_layer("Output_Layer")(decoder_layer2)
decoder_model = Model(inputs=decoder_input, outputs=decoder_output, name="Decoder")

In [4]:
print(autoencoder.summary())

Model: "Autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input_Layer (InputLayer)        │ (None, 33)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Encoder_Hidden_1 (Dense)        │ (None, 64)             │         2,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Encoder_Hidden_2 (Dense)        │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Latent_Space (Dense)            │ (None, 7)              │           231 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Decoder_Hidden_1 (Dense)        │ (None, 32)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Decoder_Hidden_2 (Dense)        │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output_Layer (Dense)            │ (None, 33)             │         2,145 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,000 (35.16 KB)

 Trainable params: 9,000 (35.16 KB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
class print_training_on_text_every_10_epochs_Callback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        with open('log_epoch.log', "a") as writefile:  # put log_path here
            if (int(epoch) % 100) == 0:
                s = f"Epoch: {epoch:>3}" + f" | Loss: {logs['loss']:.4e}" + f" | Validation loss: {logs['val_loss']:.4e}\n"
                writefile.write(s)

# Training
history = autoencoder.fit(real_data.values, real_data.values, epochs=5000, batch_size=1024, validation_split=0.2, verbose=0, callbacks=[print_training_on_text_every_10_epochs_Callback()])

In [ ]:
# Generate latent representations
latent_representations = encoder_model.predict(real_data.values)

from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ks_2samp

# Fit PCA on the latent representations
pca = PCA()
pca_latent = pca.fit_transform(latent_representations)

n_components = np.argmax(np.cumsum(pca.explained_variance_ratio_) >= 0.95) + 1
pca = PCA(n_components=n_components)
pca_latent_reduced = pca.fit_transform(latent_representations)

In [9]:
# save model
import pickle

with open('pca.pkl', 'wb') as f:
    pickle.dump(pca, f)

with open(file='ss.pkl', mode='wb') as f:
    pickle.dump(ss, f)

with open(file='diagnosis_le.pkl', mode='wb') as f:
    pickle.dump(diagnosis_le, f)

with open(file='ethnicity_le.pkl', mode='wb') as f:
    pickle.dump(ethnicity_le, f)

np.save(file='pca_latent_reduced.npy', arr=pca_latent_reduced, allow_pickle=True)

decoder_model.save('decoder_model.keras')

## Generate new samples

In [1]:
import pickle
import numpy as np
from tensorflow.keras.models import load_model

with open('diagnosis_le.pkl', 'rb') as f:
    diagnosis_le = pickle.load(f)

with open('ethnicity_le.pkl', 'rb') as f:
    ethnicity_le = pickle.load(f)

with open('ss.pkl', 'rb') as f:
    ss = pickle.load(f)

with open('pca.pkl', 'rb') as f:
    pca = pickle.load(f)

pca_latent_reduced = np.load('pca_latent_reduced.npy')

decoder_model = load_model('decoder_model.keras', compile=False)

2025-02-10 14:29:14.441097: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-10 14:29:14.461797: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739197754.485957 1291651 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739197754.493061 1291651 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-10 14:29:14.517811: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
import pandas as pd

real_data = pd.read_csv('../data/Cancer_Data_new_cols.csv')
real_data.drop(columns=['id'], inplace=True)

N_NEW_SAMPLES = 600

pca_samples = pca_latent_reduced[np.random.choice(pca_latent_reduced.shape[0], N_NEW_SAMPLES, replace=True)]

# Transform back to the original latent space
latent_samples_from_pca = pca.inverse_transform(pca_samples)

# Decode the new samples into the original feature space
generated_data = decoder_model.predict(latent_samples_from_pca)
generated_data = ss.inverse_transform(generated_data)

df_gen = pd.DataFrame(generated_data, columns=real_data.columns)
df_gen['diagnosis'] = (df_gen['diagnosis'] >= 0.5).astype(int)
df_gen['diagnosis'] = diagnosis_le.inverse_transform(df_gen['diagnosis'])

df_gen['ethnicity'] = (df_gen['ethnicity'] >= 0.5).astype(int) # if ethnicity is 2-class
df_gen['ethnicity'] = ethnicity_le.inverse_transform(df_gen['ethnicity'])

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [6]:
print(real_data.columns)

Index(['diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'age', 'ethnicity'],
      dtype='object')


In [3]:
print(df_gen.head())

  diagnosis  radius_mean  texture_mean  perimeter_mean    area_mean  \
0         B     9.256016     16.271925       58.001549   248.294250   
1         B    11.132212     18.142189       70.591606   385.240723   
2         B    10.261774     14.862250       64.489151   322.204132   
3         B    12.721702     18.552351       81.808151   492.049652   
4         M    22.789310     22.430492      158.556122  1380.030884   

   smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
0         0.087032          0.051024        0.012326             0.006140   
1         0.088617          0.051988        0.010331             0.010147   
2         0.093492          0.051827        0.001384             0.008625   
3         0.096455          0.099640        0.054149             0.037348   
4         0.110759          0.292215        0.331844             0.128697   

   symmetry_mean  ...  perimeter_worst   area_worst  smoothness_worst  \
0       0.179213  ...        64.11134

### TODO: handle etnicity like diagnosis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def correlation_matrix(real_data, synthetic_data, method='spearman'):
    real_data.drop(columns=['diagnosis', 'ethnicity'], inplace=True)
    synthetic_data.drop(columns=['diagnosis', 'ethnicity'], inplace=True)

    plt.figure(figsize=(25, 10))
    heatmap = sns.heatmap(real_data.corr(method=method), cmap="coolwarm", vmin=-1, vmax=1, annot=True)
    heatmap.set_title('Correlation Heatmap - Real data', fontdict={'fontsize':12}, pad=12)
    plt.show()

    plt.figure(figsize=(25, 10))
    heatmap = sns.heatmap(synthetic_data.corr(method=method), cmap="coolwarm", vmin=-1, vmax=1, annot=True)
    heatmap.set_title('Correlation Heatmap - Synth Data', fontdict={'fontsize':12}, pad=12)
    plt.show()


correlation_matrix(real_data, df_gen, method='spearman')

In [ ]:
import matplotlib.pyplot as plt

# Count occurrences of each category
counts1 = real_data['diagnosis'].value_counts(normalize=True)
counts2 = df_gen['diagnosis'].value_counts(normalize=True)

# Align counts for consistency (in case one category is missing)
categories = ['M', 'B']
counts1 = counts1.reindex(categories, fill_value=0)
counts2 = counts2.reindex(categories, fill_value=0)

# Plot side-by-side bar plots
x = np.arange(len(categories))  # Category positions
width = 0.35  # Bar width

fig, ax = plt.subplots()
bars1 = ax.bar(x - width/2, counts1, width, label='Real Data')
bars2 = ax.bar(x + width/2, counts2, width, label='Gen Data')

# Add labels, title, and legend
ax.set_xlabel('Category')
ax.set_ylabel('Frequency')
ax.set_title('Frequency of Diagnosis in Two Datasets')
ax.set_xticks(x)
ax.set_xticklabels(categories)
ax.legend()

# Show the plot
plt.savefig('/data/aiact/autoencoder/figures/diagnosis.png')
plt.show()


In [ ]:
#print(real_data.values.shape)
print(df_gen.shape)

print(df_gen.head())

In [ ]:
# Visualization and comparison
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
import sys
import os
sys.path.append(os.path.abspath('..'))
from evaluation_metrics import similarity_sample2sample
import pandas as pd

# real_data['diagnosis'] = le.transform(real_data['diagnosis'].astype(str))
# df_gen['diagnosis'] = le.transform(df_gen['diagnosis'].astype(str))

cols = list(real_data.columns)
cols.remove('diagnosis')
for i, col in enumerate(cols, start=1):
    plt.figure(figsize=(10, 5))
    
    # Original feature distribution
    sns.kdeplot(real_data.values[:, i], label="Original Data", fill=True, alpha=0.5, color="blue")
    
    # Generated feature distribution
    sns.kdeplot(df_gen.values[:, i], label="Generated Data", fill=True, alpha=0.5, color="orange")
    
    ks_stat = similarity_sample2sample(real_data.values[:, i], df_gen.values[:, i])
    # Plot details
    plt.title(f"Feature {col} - KS Statistic = {ks_stat:.4f}")
    plt.xlabel(f"Feature {col}")
    plt.ylabel("Density")
    plt.legend()
    plt.savefig(f'/data/aiact/autoencoder/figures/{col}.png')
    plt.show()
    

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('..'))
from evaluation_metrics import similarity_distribution2distribution, similarity_nonlinear_distribution2distribution

score = similarity_distribution2distribution(real_data.drop(columns=['diagnosis']), df_gen.drop(columns=['diagnosis'])).item()
print(f"{score = :.2f}")

score_nonlinear = similarity_nonlinear_distribution2distribution(real_data.drop(columns=['diagnosis']).values, df_gen.drop(columns=['diagnosis']).values)
print(f"{score_nonlinear = :.2f}")